In [22]:
pip install torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Load the CSV files
train_df = pd.read_csv('../input/mnist-data/mnist_train.csv')
test_df = pd.read_csv('../input/mnist-data/mnist_test.csv')

# Separate labels and features
train_labels = train_df.iloc[:, 0].values
train_data = train_df.iloc[:, 1:].values / 255.0  # Normalize pixel values

# Convert to PyTorch tensors
train_data = torch.tensor(train_data, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)

# Split the training data into train and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Convert to DataLoader for batching
train_dataset = TensorDataset(train_data, train_labels)
val_dataset = TensorDataset(val_data, val_labels)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Test data
test_data = test_df.values / 255.0
test_data = torch.tensor(test_data, dtype=torch.float32)


In [24]:
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # First fully connected layer
        self.fc2 = nn.Linear(128, 64)     # Second fully connected layer
        self.fc3 = nn.Linear(64, 10)      # Output layer (10 classes for digits 0-9)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        x = torch.relu(self.fc1(x))  # Apply ReLU activation
        x = torch.relu(self.fc2(x))  # Apply ReLU activation
        x = self.fc3(x)  # Output logits
        return x

# Create the model
model = SimpleNN()


In [25]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [26]:
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients from previous step
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

    # Validation accuracy
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculations for evaluation
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Epoch 1/10, Loss: 0.39166268406808374
Validation Accuracy: 94.14%
Epoch 2/10, Loss: 0.17391014561553797
Validation Accuracy: 95.46%
Epoch 3/10, Loss: 0.12180733296523491
Validation Accuracy: 96.31%
Epoch 4/10, Loss: 0.09007902155692378
Validation Accuracy: 96.88%
Epoch 5/10, Loss: 0.06780637302311758
Validation Accuracy: 96.81%
Epoch 6/10, Loss: 0.05537431463071456
Validation Accuracy: 97.12%
Epoch 7/10, Loss: 0.042307561668256916
Validation Accuracy: 97.42%
Epoch 8/10, Loss: 0.0333274762838458
Validation Accuracy: 97.15%
Epoch 9/10, Loss: 0.028380133412157497
Validation Accuracy: 97.67%
Epoch 10/10, Loss: 0.023388096981720688
Validation Accuracy: 97.60%


In [29]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculations for evaluation
    for images, labels in val_loader:
        images = images.view(-1, 28*28)  # Flatten the test images
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get predictions (index of max logit)
        total += labels.size(0)  # Total number of test samples
        correct += (predicted == labels).sum().item()  # Count correct predictions

# Calculate and print accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 97.60%
